# Embeddings con LSTM



In [1]:
import torch
import torch.nn as nn
from collections import Counter
import random
from itertools import chain
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
cd ..

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal


In [3]:
from helpers.boletines import get_sentences

In [4]:
cd Embeddings/

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal\Embeddings


## Definición de constantes

In [5]:
# Número de GPUs disponibles. Usar 0 para modo CPU.
ngpu = 1

# Semilla a usar en los generadores de números aleatorios
SEED = 42069
# SEED = random.randint(1, 10000) # En caso de requerir más resultados
random.seed(SEED)
torch.manual_seed(SEED)

print("Random Seed: ", SEED)

Random Seed:  42069


Selecciono el tipo de dispositivo a utilizar (gpu o cpu)

In [6]:
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

## Obtener corpus

In [7]:
uri = open('./../mongo_uri.txt', 'r', encoding='utf-8').read()

In [8]:
sentences = get_sentences(uri)

RAC
1. De_la_mano_de jóvenes innovadores , comienza Ricardo_Anaya el camino hacia la Presidencia_de_la_República .
377. Finalmente , sostuvo que López_Obrador es “ el espanta inversiones ” , porque su actitud provoca que estas se vayan , lo cual es muy delicado , porque cuando no hay inversión no crece la economía y no se generan empleos .
753. “ Sí , necesitamos cambiar ciertas piezas de el modelo económico , estamos en un momento crítico , donde tenemos que elegir entre volver a el pasado a_través_de un cómic o construir un futuro más incluyente , igualitario , a_partir_de propuestas series , razonadas , en donde los cómo están claros y los para qué , son parte principal de lo que se pretende hacer ” , enfatizó a el contrastar con la propuesta que hace López_Obrador .
1129. Yo quiero paz para Jalisco , yo quiero paz para México , y esa paz la voy a construir con las mujeres , esa paz la vamos a construir todos juntos ” , enfatizó .

JAMK
1. José_Antonio_Meade fue recibido en Culiacán

Desenvuelvo los grupos de enunciados en un único arreglo con todos los enunciados de todos los candidatos.

In [9]:
corpus = list(chain(*sentences.values()))

Paso los tokens a minúsculas para reducir el tamaño del vocabulario

In [10]:
corpus = [[w.lower() for w in sent] for sent in corpus]
print(f'El corpus consta de {len(corpus)} enunciados.')

El corpus consta de 6889 enunciados.


Selecciono k oraciones de prueba para el modelo

In [11]:
k = 1000
sentences = random.sample(corpus, k=k)
print('\n'.join([' '.join(sent) for sent in sentences[::k//10]]))

meade expuso que mientras los migrantes mexicanos contribuyen a la prosperidad y desarrollo de la sociedad y economía estadounidenses , las armas y el dinero en efectivo que ingresan desde ee.uu. a méxico caen en_manos_de delincuentes , lo que alimenta la inseguridad y la violencia en nuestro país .
pidió tener confianza , porque no les va a fallar , no va a quedar les mal , ni se decepcionará , tiene tres principios que lo guían : no mentir , no robar y no traicionar a el pueblo de méxico y se llevará_a_cabo la cuarta transformación de la vida pública de méxico , sin violencia , de manera pacífica .
pidió que el voto sea parejo por los candidatos de la coalición “ juntos_haremos_historia ” , porque ahora se repite mucho esa cantaleta de : “ vota por el peje , pero para la gobernatura no votes por lomelí o no votes por maría_antonia_cárdenas_para_el_senado ” .
comenta que espera que los otros candidatos se tranquilicen , que se serenen , es normal , están a el final de la campaña , tod

## Indexar Corpus

Cuento los tokens en todas las oraciones e imprimo los 40 más frecuentes.

In [12]:
freq = sum([Counter(sent) for sent in sentences], Counter())
print(freq.most_common()[:-40:-1])

[('toma', 1), ('recreativas', 1), ('diversas', 1), ('deportivos', 1), ('llaman', 1), ('layda_sansores', 1), ('acusación', 1), ('conservadurismo', 1), ('apoyado', 1), ('desesperación', 1), ('enfilando', 1), ('amparo', 1), ('medrando', 1), ('enemigos', 1), ('bloqueó', 1), ('crea', 1), ('preocupó', 1), ('hacendado', 1), ('apóstol', 1), ('admirador', 1), ('kilo_de_ayuda', 1), ('asociación', 1), ('centros_para_el_desarrollo_infantil_temprano', 1), ('acambay', 1), ('bebé', 1), ('pacto_por_la_primera_infancia', 1), ('radical', 1), ('candidata', 1), ('deficiencias', 1), ('involucrada', 1), ('mención', 1), ('obran', 1), ('desviar', 1), ('pretende', 1), ('infundadas', 1), ('predicar', 1), ('artes', 1), ('disfrute', 1), ('formarán_parte', 1)]


In [13]:
num_tokens = len(freq.keys())
num_tokens

5102

Agrego un diccionario para pasar de palabra a índice numérico

In [14]:
w_to_index = {
    w : ix
    for ix, (w, freq) in enumerate(freq.most_common())
    if freq > 1 # No toma en cuenta los hapax
}

In [15]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = num_tokens + 1
ixEOS = num_tokens + 2
ixUNK = num_tokens + 3

In [16]:
w_to_index[BOS] = ixBOS
w_to_index[EOS] = ixEOS
w_to_index[UNK] = ixUNK

Creo el diccionario inverso, para convertir de índices a palabras

In [17]:
index_to_w = [ w for w, ix in w_to_index.items() ]

In [18]:
# tests = random.sample(w_to_index.keys(), k=10)
# for w in tests:
#     ix = w_to_index[w]
#     assert index_to_w[ix] == w

Indexo todo el corpus

In [20]:
def w_to_index_unk(w):
    """
    Le asigna el token UNK a palabras que no aparezcan en el corpus
    """
    try:
        return w_to_index[w] 
    except KeyError:
        return ixUNK

In [39]:
sentences_ix = [
    [ w_to_index_unk(w) for w in sent ] 
    for sent in sentences
]

X = [ np.asarray(sent[:-1]) for sent in sentences_ix ]
Y = [ np.asarray(sent[1:]) for sent in sentences_ix ] 

print('Ejemplos X')
print(X[:3])
print('Ejemplos Y')
print(Y[:3])

Ejemplos X
[array([  66,  260,    3,  715,    9,  595,   86, 5105,    5,    4, 1100,
          7,  108,    1,    4,  218,    7,  236, 5105,    0,   18,  596,
          7,    2,  123,    8,  294,    3, 5105,  109, 1511,    5,   24,
       5105, 5105,  716,    0,   20,    3, 5105,    4,  237,    7,    4,
         87,    8,   88,   28]), array([   5,   78,    1,    9,  104,   54,   47,    4, 5105,  133,   89,
         12, 1101,    8,   50,  326,    0,   39,   90,  597,   35,   14,
        867,    3,   43,   22, 1512,   11,    3, 1513,    0,   12,  868,
        869,  598,  717,    0,   10, 5105,   22, 5105,    0,   20, 5105,
          7,   41,    3, 1102, 1514,   15]), array([  10,  168,    1,    3,    8,   73,    2,   27,   12,  510,    9,
        870,    8,  599, 1515,    0,   12,  510,    2,  600,    0,   96,
          2, 5105,    0,   12,   10, 1516, 5105,    0,   37,    5,  601,
          0,    5, 1517,   20,    3,   10,  718,    0,   12,   10, 5105,
          8,   20,    3,   16, 510

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

Mando los vectores de entrada y salida a tensores en gpu

In [41]:
def to_pytorch_tensor(list_of_lists):
    return [
        torch.from_numpy(l).float().to(device)
        for l in list_of_lists
    ]

In [42]:
X_train = to_pytorch_tensor(X_train)
X_test = to_pytorch_tensor(X_test)
Y_train = to_pytorch_tensor(Y_train)
Y_test = to_pytorch_tensor(Y_test)

## Modelo

### 1. Capa de embedding

### 2. Capa oculta

### 3. Capa de salida

Defino las variables para la red neuronal

In [45]:
# Dimensión de entrada (one-hot), tamaño del vocabulario
D_in = len(index_to_w)

# Dimensión de la capa de embedding
D_emb = 32

# Dimensión de la capa lstm
D_lstm = 16

# Dimensión de la capa de salida
D_out = D_in

# Épocas de entrenamiento
num_epochs = 10

# Learning rate
lr = 0.001

# Betas para Adam
beta1 = 0.0002
beta2 = 0.999

In [24]:
class Model(nn.Module):
    def __init__(self, ngpu):
        super(Model, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb),
            nn.LSTMCell(input_size=D_emb, hidden_size=D_lstm, bias=True),
            nn.Linear(in_features=D_lstm, out_features=D_out, bias=True),
            nn.Softmax(dim=D_out)
        )

    def forward(self):
        return self.main(input)

In [25]:
model = Model(ngpu).to(device)
model

Model(
  (main): Sequential(
    (0): Embedding(2278, 32)
    (1): LSTMCell(32, 16)
    (2): Linear(in_features=16, out_features=2278, bias=True)
    (3): Softmax(dim=2278)
  )
)

In [26]:
# Inicialización de pesos
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Embedding') != -1:
        # Regularizo los pesos
        n = m.num_embeddings
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
    elif classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.fill_(0)
    elif classname.find('LSTMCell') != -1:
        n = m.input_size
        y = 1.0/np.sqrt(n)
        m.weight_ih.data.uniform_(-y, y)
        m.weight_hh.data.uniform_(-y, y)
        m.bias_ih.data.fill_(0)
        m.bias_hh.data.fill_(0)

In [27]:
model.apply(weights_init)

Model(
  (main): Sequential(
    (0): Embedding(2278, 32)
    (1): LSTMCell(32, 16)
    (2): Linear(in_features=16, out_features=2278, bias=True)
    (3): Softmax(dim=2278)
  )
)

Entropía cruzada como optmizador y SGD como optimizador

In [48]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2))

### Entrenamiento

In [49]:
for epoch in range(num_epochs):
    for x, y in zip(X_train, Y_train):
        # Limpiamos gradientes acumulados
        model.zero_grad()

        # Forward
        pred = model(x)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch:>12} Loss: {loss}')


TypeError: forward() takes 1 positional argument but 2 were given